# Gaze Visualiser
This part of the notebook takes in the exported data from the gaze analyser and displays them as an interactive graph to closely interpret the data. This uses the following files:
- `gaze.csv`
- `objects.csv`
- `regions.csv`
- `experiment_info.json`
- `ttc.csv`

In [17]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings

matplotlib.use('TkAgg')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# File Paths
regions_filepath = 'visualiser_source/regions.csv'
gaze_filepath = 'visualiser_source/gaze.csv'
objects_filepath = 'visualiser_source/objects.csv'
experiment_info_filepath = 'visualiser_source/experiment_info.json'
ttc_filepath = 'visualiser_source/ttc.csv'


# Contrasting Colours
CONTRASTING_COLURS = ['#011627', '#2ec4b6', '#e71d36', '#ff9f1c']
current_colour_index = 0

Reading the CSV files as dataframes, and the experiment information a dictionary:

In [18]:
def open_files(gaze, objects, regions, ttc, experiment_info):
    gaze_df = pd.read_csv(gaze)
    objects_df = pd.read_csv(objects)
    regions_df = pd.read_csv(regions)
    ttc_df = pd.read_csv(ttc)

    with open(experiment_info, 'r') as file:
        data = json.load(file)
        exp_info = {
            'Start': data.get('Start'),
            'End': data.get('End')
        }

    return gaze_df, objects_df, regions_df, ttc_df, exp_info

gaze, objects, regions, ttc_df, exp_info = open_files(gaze_filepath, objects_filepath, regions_filepath, ttc_filepath, experiment_info_filepath)

## Plot TTC
Takes the object speed, and plots the TTC values 

In [19]:
def plot_ttc(ttc_df):
    figure, ax = plt.subplots(figsize=(16, 8))
    ax.scatter(ttc_df['Object Speed'], ttc_df['Time to Contact'])
    ax.set_title('Object Speed vs Time to Contact')
    ax.set_xlabel('Object Speed')
    ax.set_ylabel('Time to Contact')
    ax.grid(True)
    figure.show()

# plot_ttc(ttc_df)

## Displaying Gaze Positions
This is to visualise the difference smoothed and unsmoothed version of `norm_pos_x` and  `norm_pos_y`:

In [20]:
def plot_norm_positions(df):
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    axs[0].plot(df['time'], df['norm_pos_x'], label='Original norm_pos_x')
    axs[0].plot(df['time'], df['smoothed_norm_pos_x'], label=f'Smoothed norm_pos_x (window={10})', linestyle='--')
    axs[0].set_title('Norm_pos_x')

    axs[1].plot(df['time'], df['norm_pos_y'], label='Original norm_pos_y')
    axs[1].plot(df['time'], df['smoothed_norm_pos_y'], label=f'Smoothed norm_pos_y (window={10})', linestyle='--')
    axs[1].set_title('Norm_pos_y')

    for ax in axs:
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Position')
        ax.legend()
    fig.show()

# plot_norm_positions(gaze_df)

## Plotting the interception graph
The following functions looks at taking each of the dataframe, and plotting the graph according to using smoothed or unsmoothed positions.

### Angular Distance agaisnt Time Graph
The first graph plots the main graph that acts as the basis representation for gaze positions:

In [21]:
def plot_angular_distance(fig, ax, df, use_smooth=True, show_both=False):

    if show_both:
        ax.plot(df['time'], df['angular_distance'], label='Gaze Angular Distance',)
        ax.plot(df['time'], df['smoothed_angular_distance'], label=f'Angular Distance', linestyle='--')
    else:
        if use_smooth:
            ax.plot(df['time'], df['smoothed_angular_distance'], label=f'Angular Distance', color='orange')
        else:
            ax.plot(df['time'], df['angular_distance'], label='Gaze Angular Distance', color='orange')

    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angular Distance (rad)', color='black')
    ax.legend(loc='lower right')
    fig.suptitle('Interception Experiment')
    fig.tight_layout()

# plot_angular_distance(figure, ax, gaze)

## Visualising Eye Movements
Within the dataframe, there is a field called `movement_type` where we identify which one of the three eye movements (Saccade, Fixations, and Smooth Pursuit) base on velocities and provided csv file from Pupil Lab.

### Regions of Saccades

In [22]:
def plot_saccades(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type= 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type = 'movement_type'

    distances = df[source_distance]
    saccade_regions = df[movement_type] == 'Saccades'

    ax.plot(df['time'], np.where(saccade_regions, distances, np.nan), color='red', label='Saccade Line')
    ax.legend(loc='lower right')

### Regions of Smooth Pursuit

In [23]:
def plot_smooth_pursuits(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type_column = 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type_column = 'movement_type'

    distances = df[source_distance]
    smooth_pursuit_regions = df[movement_type_column] == 'Smooth Pursuit'

    # Plot for Smooth Pursuit Data
    ax.plot(df['time'], np.where(smooth_pursuit_regions, distances, np.nan), color='green', label='Smooth Pursuit')
    ax.legend(loc='upper left')

### Regions of Fixation

In [24]:
def plot_fixations(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type= 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type = 'movement_type'

    distances = df[source_distance]
    fixation_regions = df[movement_type] == 'Fixation'

    ax.plot(df['time'], np.where(fixation_regions, distances, np.nan), color='blue', label='Fixation')
    ax.legend(loc='lower right')

### Combining the identification function:
This would combine each of the function and allow the user control on what they wish to present on the graph

In [25]:
def plot_data(figure, ax, gaze_df, use_smooth=True, fixations=False, smooth_pursuits=False, saccades=False):
    plot_angular_distance(figure, ax, gaze_df, use_smooth=use_smooth)
    if fixations:
        plot_fixations(ax, gaze_df, use_smooth=use_smooth)
    
    if smooth_pursuits:
        plot_smooth_pursuits(ax, gaze_df, use_smooth=use_smooth)

    if saccades:
        plot_saccades(ax, gaze_df, use_smooth=use_smooth)

    return figure, ax

### Visualising annotations
Annotations consists of multiple types. These are: Spawning, Intercepted, Looking At, Experiment End/Start. To simplify this approach, the analyser exports two files `regions.csv` and `objects.csv` which already has information which lets us display the regions and object information on the graph:

#### Adjusting and showing experiment start/end
Due to the recording starting before the experiment starts and delays before ending the experiment, we want to indicate which part of the graph is most relevant at first

In [26]:
def plot_experiment_lines(ax, experiment_info):
    start = experiment_info['Start']
    end = experiment_info['End']
    ax.axvline(x=start, color='black', linestyle='--')
    ax.axvline(x=end, color='black', linestyle='--')
    ax.set_xlim(start, end) # Adjust the graph to display between the lines when the graph appears

#### Representing Objects
To represent objects, each has been assigned a HEX code for a colour. They have been chosen as adjacent colours are contrasting for easy visualisation and avoid generating colours that might be close to each other:


In [27]:
def choose_colour():
    global current_colour_index
    colour = CONTRASTING_COLURS[current_colour_index] # Gets the colour in the current index
    current_colour_index = (current_colour_index + 1) % len(CONTRASTING_COLURS) # Increments the current index
    return colour

In [28]:
def assign_colour(objects_df):
    object_colours = {}
    for _, row in objects_df.iterrows():
        object_id = int(row['id'])
        object_colours[object_id] = choose_colour()
        # print(f'{object_id} with colour {object_colours[object_id]}')
    return object_colours

# object_colours = assign_colour(objects)

Once each object has been given a colour, we can start plotting the regions of observation, and spawn/intercept lines

In [29]:
def plot_objectLines(ax, objects_df, object_colours):
    for _, row in objects_df.iterrows():

        object_id = int(row['id'])
        line_colour = object_colours[object_id]

        spawn_time = row['Spawning']
        ax.axvline(x=spawn_time, color=line_colour, linestyle='--', alpha=0.7)
        ax.text(spawn_time, ax.get_ylim()[0] + 0.02, f'Spawned {object_id} at {spawn_time:.2f}', rotation=90, va='bottom', ha='right', color='black')

        intercepted_time = row['Intercepted']
        if intercepted_time != pd.NaT:
            ax.axvline(x=intercepted_time, color=line_colour, linestyle='--', alpha=0.7)
            ax.text(intercepted_time, ax.get_ylim()[1] - 0.02, f'Intercepted {object_id} at {intercepted_time:.2f}', rotation=90, va='top', ha='right', color='black')

# plot_objectLines(ax, objects, object_colours)

Plotting the observation across the experiment as regions on the graph

In [30]:
def plot_observations(ax, regions_df, object_colours):
    for _, row in regions_df.iterrows():
        object_id = row['id']
        colour = object_colours[object_id]
        start = row['start_time']
        end = row['end_time']

        ax.axvspan(start, end, color=colour, alpha=0.2)

# plot_observations(ax, regions, object_colours)

#### Combined Annotation functions
This function combines all the annotation related files

In [31]:
def plot_annotations(ax, regions_df, objects_df, experiment_info):
    object_colours = assign_colour(objects_df)
    plot_experiment_lines(ax, experiment_info)
    plot_objectLines(ax, objects_df, object_colours)
    plot_observations(ax, regions_df, object_colours)

## Displaying the graph

In [32]:
figure, ax = plt.subplots(figsize=(16, 8))
plot_data(figure, ax, gaze, use_smooth=True, fixations=True, smooth_pursuits=True, saccades=True)
# plot_ttc(ttc_df)
plot_annotations(ax, regions, objects, exp_info)

figure.show()